In [7]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from nltk.tokenize import sent_tokenize
import nltk
import time
import pickle
from torch.utils.data import DataLoader


path = "/global/cscratch1/sd/ajaybati/pickles/"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [4]:
maxdict = {}
x=0
total = 0
a=0
for note in notes_df2["TEXT"]:
    x+=1
    start = time.time()
    for sent in sent_tokenize(note):
        try:
            encdict = tokenizer.encode_plus(sent, return_tensors='pt')
            length = len(encdict['input_ids'][0])
            if length not in maxdict:
                maxdict[length] = 1
            else:
                maxdict[length]+=1
        except:
            pass
    end=time.time()
    total+=(end-start)
    if x%1000==0:
        a+=1
        print(x,"/",17)
        print(total*1/x*(len(notes_df2["TEXT"])-1000*a)/3600)
    time.sleep(2)




maxdf = pd.DataFrame(data = maxdict).set_index(maxdict.keys())

with open(path+'countsentlength.pickle','rb') as f:
    maxdf = pickle.load(f)

maxdf = pd.DataFrame({'Count Numbers':list(maxdict.keys()),'Frequency':list(maxdict.values())})
maxdf[maxdf['Count Numbers']<128]['Frequency'].sum()/maxdf['Frequency'].sum()

maxdf['Frequency'].sum()

In [10]:
noteevents = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv")

# salience_patient = pd.read_pickle(path+"salience_patient.pickle")



final_df = pd.read_pickle(path+"final_df.pickle")
notes_df = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = final_df[["HADM_ID","SUBJECT_ID","DOB"]].drop_duplicates().merge(notes_df, on=["SUBJECT_ID","HADM_ID"], how="right")

notes = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = pd.merge(final_df[["SUBJECT_ID","HADM_ID","DOB"]],notes.drop_duplicates(subset = ["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME"]), on=["SUBJECT_ID","HADM_ID"], how = "left")
notes_df = notes_df.drop_duplicates(subset='TEXT')
notes_df2 = notes_df[notes_df["CATEGORY"]=="Discharge summary"]
notes_df2

In [ ]:
#data collection
import random
class textLoader(Dataset):
    def __init__(self,transform = None, yes = True):
        self.input_ids_all = []
        self.attention_masks_all = []
        self.text = notes_df2["TEXT"].tolist()
        self.samples = len(self.text)
        total = 0
        x=0
        for note in self.text:
            start = time.time()
            try:
                for sent in sent_tokenize(note):
                    
                    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,
                        truncation = True,# Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
                    # Add the encoded sentence to the list.    
                    input_ids = encoded_dict['input_ids']
                    attention_masks = encoded_dict['attention_mask']

                    if yes:
                        input_ids_real = []
                        for index,word in enumerate(input_ids[0]):
                            if int(word) != 101 and int(word) != 102:
                                rando = random.random()
                                random.seed()
                                if rando < 0.2 and int(word)!=0:
                                    input_ids_real.append(103)
                                else:
                                    input_ids_real.append(int(word))
                            else:
                                input_ids_real.append(int(word))
                        input_ids_real = torch.tensor(input_ids_real).view(1,64)

                        self.input_ids_all.append(input_ids_real)
                        self.attention_masks_all.append(attention_masks)
                    else:

                        self.input_ids_all.append(input_ids)
                        self.attention_masks_all.append(attention_masks)
                        
            except Exception as e:
                print(str(e), input_ids)
            end = time.time()
            if((self.text.index(note)+1)%10000==0):
                x+=1
                total+=(end-start)
                print((self.text.index(note)+1)/len(self.text), total/(x))
       
            
    def __getitem__(self,index):
        return self.input_ids_all[index],self.attention_masks_all[index]
    
    
    def __len__(self):
        return self.samples

    

data_2 = textLoader(yes = True)
print("done")
data_input_randomized = torch.cat(tuple(data_2.input_ids_all),dim = 0)
attention_masks_randomized = torch.cat(tuple(data_2.attention_masks_all),dim = 0)
print("save 1")
torch.save(data_input_randomized,"pickles/input_ids_randomized30.pickle")
print("save 2")
torch.save(attention_masks_randomized,"pickles/attention_masks_randomized30.pickle")
print("done")

In [ ]:
input_ids_all = torch.load(path+"DSmodeldata128.pickle")
attention_masks_all = torch.load(path+"DSmodeldataAM128.pickle")
input_ids_real = torch.load(path+"DSmodeldatareal128.pickle")
mask_indices = []
x=0
for sent in input_ids_all:
    start=time.time()
    sentence = list(sent)
    mask = []
    for word in sentence:
        if word==103:
            mask.append(index)
    mask_indices.append(mask)
    end = time.time()
    x+=1
    if x%500000==0:
        print("="*20+" "+str(x)+"/8286324"+"="*20, len(mask)/len(masked_indices))

mask_indices_tensor = []
number=0
for mask in mask_indices:
    number+=1
    padded = []
    try:
        a = time.time()
        if len(mask)==0:
            for x in range(135):
                padded.append(0)
        else:
            padded = padded+mask
            for x in range(135-len(padded)):
                padded.append(0)
        mask_indices_tensor.append(torch.tensor(padded).view(1,135))
        if ((number%500000)==0):
            print("="*20+str(number)+"/2197122"+"="*20)
            print(time.time()-a)
            
    except Exception as e:
        print(str(e))
        print(type(mask))
        print(padded)
mask_indices_tensor = torch.cat(tuple(mask_indices_tensor),0)
torch.save(mask_indices_tensor,path+'mask_indices_tensorDS128.pickle')

mask_indices_tensor = torch.load(path+"mask_indices_tensorDS128.pickle")

In [5]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_all, attention_masks_all, input_ids_real, mask_indices_tensor)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,977,409 training samples
219,713 validation samples


In [6]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
torch.save(train_dataloader,path+"train_dataloaderDS128.pickle")
torch.save(validation_dataloader,path+"validation_dataloaderDS128.pickle")


In [ ]:
#testing
masked_index = 12
note_num = 0
sent = 1
note = data.input_ids_all[note_num]
retokenize = [tokenizer.convert_ids_to_tokens(sent) for sent in note]
real_sent = [tokenizer.convert_ids_to_tokens(sent) for sent in note]
retokenize[sent][masked_index] = '[MASK]'
start = retokenize[sent].index('[CLS]')+1
end = retokenize[sent].index('[SEP]')
retokenize = retokenize[sent][start:end]
print(retokenize)
encoded_dict = tokenizer.encode_plus(
                        retokenize,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
indexed_tokens = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']
print(indexed_tokens)

predictions = model(indexed_tokens, attention_mask = attention_mask)

predicted_index = torch.argmax(predictions[0][0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

for predic in predictions[0][0]:
    model.train()
    predicted_index = torch.argmax(predic).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print('predicted_token', predicted_token)
    print("*"*80)
    x+=1

In [ ]:
train_dataloader = torch.load(path+'train_dataloaderDS128.pickle')

In [17]:
#testing using functions
len(train_dataloader)

train_dataloader = torch.load(path+'train_dataloader.pickle')
print("done")


for step, batch in enumerate(train_dataloader):
    for bat in batch:
        print(bat)
        print("*"*20)
    break

for stuff in batch[3]:
    if list(stuff).count(torch.tensor([0]))==40:
        print("hey")

model = BertForMaskedLM.from_pretrained('bert-base-uncased')

a = "The dog loves to play with its chew toy"
encoded_dict = tokenizer.encode_plus(
                        a,                      # Sentence to encode.
                        add_special_tokens = True,
                        truncation=True,# Add '[CLS]' and '[SEP]'
                        max_length = 11,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
real = encoded_dict['input_ids']
indexed_tokens = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']
indexed_tokens= indexed_tokens
indexed_tokens[0][8] = 103


indexed_tokens

loss, predictions = model(indexed_tokens, 
                              attention_mask=attention_mask, 
                              masked_lm_labels=real)

print(loss)
print(predictions)


predicted_index = torch.argmax(predictions[0, 8]).item() #8 is one of the masked tokens index
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

top_k = 10
probs = torch.nn.functional.softmax(predictions[0, 8], dim=-1)
top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

for i, pred_idx in enumerate(top_k_indices):
    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
    token_weight = top_k_weights[i]
    print(predicted_token, token_weight)

import nltk
from nltk.translate.bleu_score import SmoothingFunction

def getSent_pred(prediction,real_labels):
    sentlist_real = []
    sep_list = []
    for sent2 in real_labels:
        tokenized = tokenizer.convert_ids_to_tokens(sent2)
        sep = tokenized.index('[SEP]')
        sep_list.append(sep)
        sentlist_real.append(tokenized[1:sep])
    
    
    sentlist_ids = []
    sentlist = []
    for sent in prediction:
        word_list = []
        for word in sent:
            word_list.append(torch.argmax(word))
        sentlist_ids.append(word_list)
    
    for index,sent in enumerate(sentlist_ids):
        sentlist.append(tokenizer.convert_ids_to_tokens(sent)[1:sep_list[index]])
    return sentlist,sentlist_real
p,r = getSent_pred(predictions,batch[2])


BLEUscore = nltk.translate.bleu_score.sentence_bleu(p[10],r[10],smoothing_function=smoothie)
BLEUscore

def bleu(p,r):
    smoothie = SmoothingFunction().method2
    bleu_list = []
    for index in range(len(p)):
        BLEUscore = nltk.translate.bleu_score.sentence_bleu(p[index],r[index],smoothing_function=smoothie)
        bleu_list.append(BLEUscore)
    return sum(bleu_list) / len(bleu_list)
bleu(p,r)

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction

for batch in train_dataloader:
    b_input_ids = batch[0]
    b_input_mask = batch[1]
    b_input_ids_real = batch[2]
    b_input_mask_ids = batch[3]
    break
    
loss, predictions = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        masked_lm_labels=b_input_ids_real)


def getSent_pred(prediction,real_labels):
    sentlist_real = []
    sep_list = []
    for sent2 in real_labels:
        tokenized = tokenizer.convert_ids_to_tokens(sent2)
        sep = tokenized.index('[SEP]')
        sep_list.append(sep)
        sentlist_real.append(tokenized[1:sep])
    
    
    sentlist_ids = []
    sentlist = []
    for sent in prediction:
        word_list = []
        for word in sent:
            word_list.append(torch.argmax(word))
        sentlist_ids.append(word_list)
    
    for index,sent in enumerate(sentlist_ids):
        sentlist.append(tokenizer.convert_ids_to_tokens(sent)[1:sep_list[index]])
    return sentlist,sentlist_real

def bleu(p,r):
    smoothie = SmoothingFunction().method2
    bleu_list = []
    for index in range(len(p)):
        BLEUscore = nltk.translate.bleu_score.sentence_bleu(p[index],r[index],smoothing_function=smoothie)
        bleu_list.append(BLEUscore)
    return sum(bleu_list) / len(bleu_list)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

def calc_accuracy(prediction, real_labels, mask_indices):
    score = 0
    total = 0
    for step,sent in enumerate(mask_indices):
        if list(sent).count(0)!=40:
            for mask in sent:
                if int(mask)!=0:
                    top_k = 10
                    probs = torch.nn.functional.softmax(predictions[step, int(mask)], dim=-1)
                    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

                    for i, pred_idx in enumerate(top_k_indices):
                        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
                        token_weight = top_k_weights[i]
                        print(predicted_token, token_weight)
                    print('='*80)
                    predicted_index = int(torch.argmax(prediction[step,int(mask)]))
                    actual = int(real_labels[step][int(mask)])
                    if bool(predicted_index==actual):
                        score+=1
                    total+=1
                else:
                    pass

        else:
            pass
    
    p,r = getSent_pred(prediction,real_labels)
    
    
    accuracy = score/total
    try:
        bscore = bleu(p,r)
    except:
        bscore = "Unfortunately, not possible"
    return accuracy, bscore 
acc, bscore = calc_accuracy(predictions, b_input_ids_real, b_input_mask_ids)
print(acc, bscore)